# Ideas
## Forecasting with Lag method
* Add results from previous rows as new features "past observations" using the pandas `shift()`.
* Make sure to add a new feature `delta_time` observations due to data split are not evenly spaced.


[https://www.bi4all.pt/en/news/en-blog/supervised-machine-learning-in-time-series-forecasting/] 

In [183]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import seaborn as sns

# Machine Learning Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor

# Model Metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error

# Model Parameters Hypertuning
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin, space_eval
from math import sqrt
from sklearn.model_selection import GridSearchCV

# Loading Data

In [184]:
# Loading Data
dataset_path = 'kaggle/input/seoul-bike-rental-ai-pro-iti'
for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input/seoul-bike-rental-ai-pro-iti\FullData.csv
kaggle/input/seoul-bike-rental-ai-pro-iti\sample_submission.csv
kaggle/input/seoul-bike-rental-ai-pro-iti\submission.csv
kaggle/input/seoul-bike-rental-ai-pro-iti\test.csv
kaggle/input/seoul-bike-rental-ai-pro-iti\train.csv


In [185]:
full_data = pd.read_csv(os.path.join(dataset_path, 'FullData.csv'), encoding='utf8')#, index_col=0)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(C)             8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(�C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

In [186]:
full_data = full_data.rename({'Rented Bike Count':'y'}, axis=1)
df_test = full_data.sample(frac=3/8, random_state=42)
df = full_data.drop(df_test.index)
print('df shape:', df.shape)
print('test shape:', df_test.shape)

df shape: (5475, 14)
test shape: (3285, 14)


In [187]:
def rename_headers(df):
    """Function removes the special character from the 2 columns: 'Temperature' and 'Dew point temperature' """
    df.rename(columns={'Temperature(�C)':'Temperature(C)', 'Dew point temperature(�C)':'Dew point temperature(C)'}, inplace=True)

rename_headers(df)
df.head()

,Date,y,Hour,Temperature(C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
5,01/12/2017,100,5,-6.4,37,1.5,2000,-18.7,0.0,0.0,0.0,Winter,No Holiday,Yes


# Preprocess Features

In [188]:
def prepare_features(df):

    """Function prepares input DataFrame for the ML model.
    It encodes and prepares the following columns:
    - `Holiday`
    - `Seasons`
    - `Date` to datetime
    - `Year`
    - `Month`
    - `Hour`

    - Drops:
        -`Dew point temperature`
    - Creates features:
        -`is_Night`
        - 'Rain_n_Snow`
    """
    df_model = df.copy()

    df_model['Holiday'] = df_model['Holiday'].apply(lambda x: 1 if x == 'Holiday' else 0)

    df_model['Date'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df.Hour, unit='h')
    df_model['WeekDay'] = df_model['Date'].dt.weekday
    df_model['Day'] = df_model.Date.dt.day
    df_model['Month'] = df_model.Date.dt.month
    df_model['Year'] = df_model.Date.dt.year

    seasons_encoding = {"Seasons": {"Winter": 0, "Spring": 1, "Summer": 2, "Autumn": 3, "Fall": 4}}
    functioning_day_encoding = {"Functioning Day": {'Yes': 1, 'No': 0}}
    df_model.replace(seasons_encoding, inplace=True)
    df_model.replace(functioning_day_encoding, inplace=True)

    # Create a feature Rain_n_Snow to create interaction between Rain and Snow
    df_model['Rain_n_Snow'] = (df_model['Rainfall(mm)'] > 2.5) & (df_model['Snowfall (cm)'] > 0.5)
    
    df_model['is_Night'] = ((df_model.Hour > 20) | (df_model.Hour < 5)).astype('int')

    # Transformations
    df_model['Snowfall (cm)'], df_model['Rainfall(mm)'] = np.log(np.clip(df['Snowfall (cm)'], 1e-7, None)), np.log(np.clip(df['Rainfall(mm)'], 1e-7, None))
    df_model['Solar Radiation (MJ/m2)'] = df['Solar Radiation (MJ/m2)'] ** 0.6
    df_model['y'] = (df['y']) ** 0.09

    # Time lag and rolling window for forecasting
    df_model.sort_values(by=['Date'], inplace=True)
    
    # lag_features = ["Temperature(C)", "Rainfall(mm)", "Visibility (10m)", "Wind speed (m/s)", "Humidity(%)", "Solar Radiation (MJ/m2)"] 
    # df_model = pd.concat(
    #     [
    #         df_model, 
    #         df_model[lag_features].rolling(4).mean().add_suffix('_rolled'), 
    #         df_model[lag_features].shift(1).add_suffix('_lag'), 
    #         df_model[lag_features].shift(2).add_suffix('_lag2')], 
    #     axis=1)

    # # Dropping
    # df_model.fillna(0, inplace=True)
    df_model.drop(columns=['Dew point temperature(C)', 'Wind speed (m/s)', 'Date'], inplace=True)

    return df_model


# Apply Feature preprocessing on Data
df_model = prepare_features(df)
display(df_model.shape)
display(df_model.head())

(5475, 17)

,y,Hour,Temperature(C),Humidity(%),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,WeekDay,Day,Month,Year,Rain_n_Snow,is_Night
1,1.613863,1,-5.5,38,2000,0.0,-16.118096,-16.118096,0,0,1,3,12,1,2017,False,1
2,1.590099,2,-6.0,39,2000,0.0,-16.118096,-16.118096,0,0,1,3,12,1,2017,False,1
3,1.522806,3,-6.2,40,2000,0.0,-16.118096,-16.118096,0,0,1,3,12,1,2017,False,1
4,1.480091,4,-6.0,36,2000,0.0,-16.118096,-16.118096,0,0,1,3,12,1,2017,False,1
5,1.513561,5,-6.4,37,2000,0.0,-16.118096,-16.118096,0,0,1,3,12,1,2017,False,0


# Split the timeseries data to train and validation

# Precition Model

In [190]:
df_train, df_val = train_test_split(df_model, test_size=0.2, random_state=42) 

# df_train = df_train.sort_values(by='Date')
# df_val = df_val.sort_values(by='Date')

X_train = df_train.drop(columns=['y'])
y_train = df_train.y

X_val = df_val.drop(columns=['y'])
y_val = df_val.y

In [197]:
space ={
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1000, dtype=int)),
    'max_depth': hp.choice('max_depth', np.arange(3, 12, 1, dtype=int)),
    'colsample_bytree': hp.uniform('colsample_bylevel', 0.2, 1.0),
    'subsample': hp.uniform('random_strength', 0.4, 1)
    }


def score(params):
    model = XGBRegressor(**params)
    
    model.fit(X_train, y_train,
               eval_set=[(X_val, y_val)],
               verbose=False)
    y_pred = model.predict(X_val).clip(0)
    score = sqrt(mean_squared_log_error(y_val ** (1 / .09), y_pred ** (1 / .09)))
    print(score)
    return {'loss': score, 'status': STATUS_OK}    
    
def optimize(trials, space):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=10)
    return best

trials = Trials()
best_params = optimize(trials, space)

# Return the best parameters

params = space_eval(space, best_params)

0.38133742741824356
0.4396701856254293
0.4546573333512381
0.4075266713393371
0.4621640131103408
0.39494809135842657
0.37822718796831434
0.47250054637891925
0.36395399314513516
0.3762889858735851
100%|██████████| 10/10 [00:21<00:00,  2.11s/trial, best loss: 0.36395399314513516]


In [198]:
xgb = XGBRegressor(n_jobs=-1, **params, random_state=42)
xgb.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_val,y_val)], verbose=False)
y_pred = np.clip(xgb.predict(X_val), a_min=0, a_max=None)
np.sqrt(mean_squared_log_error(y_val ** (1/0.09), y_pred ** (1/0.09)))

0.3728608379524561

In [199]:
# Read test csv
# df_test = pd.read_csv(os.path.join(dataset_path, 'test.csv'), encoding='utf8', index_col=0)

# Rename headers and replace diamond shapes 
rename_headers(df_test)

# Preprocess features
df_test = prepare_features(df_test)
y_true_test = df_test['y']
df_test.drop(columns=['y'], inplace=True)

# Create predictions
y_pred = xgb.predict(df_test) 
y_pred[y_pred < 0] = 0

# Add predictions column and submit csv
df_test['y'] = y_pred
# df_test['y'].to_csv(os.path.join(dataset_path, 'submission.csv'), index=True)

rmsle = np.sqrt(mean_squared_log_error(y_true_test ** (1/0.09), y_pred ** (1 / 0.09)))
print('rmsle:', rmsle)

mse = mean_squared_error(y_true_test ** (1/0.09), y_pred ** (1 / 0.09))
print('mse:',mse)

rmsle: 0.3587414587238461
mse: 45686.172031853625
